In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import psycopg2
import sqlite3
import config
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, func, inspect, Column, Integer, String, Float
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

Base = declarative_base()

In [4]:
# Read in the data
covid_data_df = pd.read_csv('covid-data.csv')
covid_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'covid-data.csv'

In [3]:
# Data types for the columns

covid_data_df.dtypes

NameError: name 'covid_data_df' is not defined

In [28]:
# Select the columns to keep

covid_data_df2 = covid_data_df[["iso_code", "continent", "location", "date", "total_cases", "total_deaths", "total_vaccinations", "people_fully_vaccinated", "population"]]
covid_data_df2.head()

,iso_code,continent,location,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated,population
0,AFG,Asia,Afghanistan,2/24/20,1.0,NaN,NaN,NaN,38928341.0
1,AFG,Asia,Afghanistan,2/25/20,1.0,NaN,NaN,NaN,38928341.0
2,AFG,Asia,Afghanistan,2/26/20,1.0,NaN,NaN,NaN,38928341.0
3,AFG,Asia,Afghanistan,2/27/20,1.0,NaN,NaN,NaN,38928341.0
4,AFG,Asia,Afghanistan,2/28/20,1.0,NaN,NaN,NaN,38928341.0


In [31]:
# Data Cleaning
covid_data_df2=covid_data_df2.fillna(0)
covid_data_df2.head()

,iso_code,continent,location,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated,population
0,AFG,Asia,Afghanistan,2/24/20,1.0,0.0,0.0,0.0,38928341.0
1,AFG,Asia,Afghanistan,2/25/20,1.0,0.0,0.0,0.0,38928341.0
2,AFG,Asia,Afghanistan,2/26/20,1.0,0.0,0.0,0.0,38928341.0
3,AFG,Asia,Afghanistan,2/27/20,1.0,0.0,0.0,0.0,38928341.0
4,AFG,Asia,Afghanistan,2/28/20,1.0,0.0,0.0,0.0,38928341.0


In [32]:
# Check any missing data
for col in covid_data_df2.columns:
    missing_data = np.mean(covid_data_df2[col].isnull())
    print('{} - {}%'.format(col, missing_data))

iso_code - 0.0%
continent - 0.0%
location - 0.0%
date - 0.0%
total_cases - 0.0%
total_deaths - 0.0%
total_vaccinations - 0.0%
people_fully_vaccinated - 0.0%
population - 0.0%


In [34]:
#Renaming the columns

covid_data_df2 = covid_data_df2.rename(
    columns={"iso_code":"Country_Code", "location":"Country"})
covid_data_df2

,Country_Code,continent,Country,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated,population
0,AFG,Asia,Afghanistan,2/24/20,1.0,0.0,0.0,0.0,38928341.0
1,AFG,Asia,Afghanistan,2/25/20,1.0,0.0,0.0,0.0,38928341.0
2,AFG,Asia,Afghanistan,2/26/20,1.0,0.0,0.0,0.0,38928341.0
3,AFG,Asia,Afghanistan,2/27/20,1.0,0.0,0.0,0.0,38928341.0
4,AFG,Asia,Afghanistan,2/28/20,1.0,0.0,0.0,0.0,38928341.0
...,...,...,...,...,...,...,...,...,...
106809,ZWE,Africa,Zimbabwe,7/29/21,105656.0,3421.0,2324034.0,730378.0,14862927.0
106810,ZWE,Africa,Zimbabwe,7/30/21,107490.0,3490.0,2375361.0,751487.0,14862927.0
106811,ZWE,Africa,Zimbabwe,7/31/21,108860.0,3532.0,2413509.0,767910.0,14862927.0
106812,ZWE,Africa,Zimbabwe,8/1/21,109546.0,3583.0,2433341.0,779229.0,14862927.0


In [35]:
covid_data_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106814 entries, 0 to 106813
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Country_Code             106814 non-null  object 
 1   continent                106814 non-null  object 
 2   Country                  106814 non-null  object 
 3   date                     106814 non-null  object 
 4   total_cases              106814 non-null  float64
 5   total_deaths             106814 non-null  float64
 6   total_vaccinations       106814 non-null  float64
 7   people_fully_vaccinated  106814 non-null  float64
 8   population               106814 non-null  float64
dtypes: float64(5), object(4)
memory usage: 7.3+ MB


In [36]:
finalcovid_data_df2 = covid_data_df2[["continent","Country","Country_Code","population","date","total_cases","total_deaths","total_vaccinations","people_fully_vaccinated"]]
finalcovid_data_df2.head()

,continent,Country,Country_Code,population,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated
0,Asia,Afghanistan,AFG,38928341.0,2/24/20,1.0,0.0,0.0,0.0
1,Asia,Afghanistan,AFG,38928341.0,2/25/20,1.0,0.0,0.0,0.0
2,Asia,Afghanistan,AFG,38928341.0,2/26/20,1.0,0.0,0.0,0.0
3,Asia,Afghanistan,AFG,38928341.0,2/27/20,1.0,0.0,0.0,0.0
4,Asia,Afghanistan,AFG,38928341.0,2/28/20,1.0,0.0,0.0,0.0


In [45]:
finalcovid_data_df2.to_csv

<bound method NDFrame.to_csv of        continent      Country Country_Code  population     date  total_cases  \
0           Asia  Afghanistan          AFG  38928341.0  2/24/20          1.0   
1           Asia  Afghanistan          AFG  38928341.0  2/25/20          1.0   
2           Asia  Afghanistan          AFG  38928341.0  2/26/20          1.0   
3           Asia  Afghanistan          AFG  38928341.0  2/27/20          1.0   
4           Asia  Afghanistan          AFG  38928341.0  2/28/20          1.0   
...          ...          ...          ...         ...      ...          ...   
106809    Africa     Zimbabwe          ZWE  14862927.0  7/29/21     105656.0   
106810    Africa     Zimbabwe          ZWE  14862927.0  7/30/21     107490.0   
106811    Africa     Zimbabwe          ZWE  14862927.0  7/31/21     108860.0   
106812    Africa     Zimbabwe          ZWE  14862927.0   8/1/21     109546.0   
106813    Africa     Zimbabwe          ZWE  14862927.0   8/2/21     110855.0   

       

### Creating class for SQL Database

In [44]:
class Covid_Data(Base):
    __tablename__ = "Covid_Data"
    ID = Column(Integer, primary_key=True)
    continent = Column(String(255))
    Country = Column(String(255))
    Country_Code = Column(Integer)
    population = Column(Integer)
    date = Column(Integer)
    total_cases = Column(String(255))
    total_deaths = Column(String(255))
    total_vaccinations = Column(String(255))
    people_fully_vaccinated = Column(Integer)

In [46]:
# Connect to local database
#Create engine connection
#engine = create_engine("sqlite:///covid.sqlite", echo=False)
engine = create_engine("sqlite:///covid.sqlite")
Base.metadata.create_all(engine)
session = Session(engine)
conn = engine.connect()

In [53]:
#Uploading covid data to sqlite database
finalcovid_data_df2.to_sql("Covid_Data", engine, if_exists="append", index=False)

In [4]:
# Test connection
con = sqlite3.connect("covid.sqlite")
query ="SELECT continent,Country,Country_Code,population,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated from Covid_Data"
country_df = pd.read_sql_query(query, con)
country_df.head()

,continent,Country,Country_Code,population,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated
0,Asia,Afghanistan,AFG,38928341,2/24/20,1.0,0.0,0.0,0
1,Asia,Afghanistan,AFG,38928341,2/25/20,1.0,0.0,0.0,0
2,Asia,Afghanistan,AFG,38928341,2/26/20,1.0,0.0,0.0,0
3,Asia,Afghanistan,AFG,38928341,2/27/20,1.0,0.0,0.0,0
4,Asia,Afghanistan,AFG,38928341,2/28/20,1.0,0.0,0.0,0
